In [ ]:
import numpy as np
import h5py
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from gw_smc_utils.plotting import set_style, lighten_colour

set_style()

In [ ]:
data_release_path = Path("../data_release/gw_smc_data_release_results/")

In [ ]:
injections = pd.read_hdf("../experiments/simulated_data/pp_tests/data/bbh_injections_uniform_chirp_mass.hdf5", "injections")
n_injections = len(injections)

In [ ]:
results_path = data_release_path / "simulated_data" / "pp_tests"
dynesty_2det_path = results_path / "dynesty_2det"
dynesty_3det_path = results_path / "dynesty_3det"
pocomc_2det_path = results_path / "pocomc_2det"
pocomc_3det_path = results_path / "pocomc_3det"

In [ ]:
network_snr_3det = injections["network_snr"]
network_snr_2det = np.sqrt(injections["H1_snr"]**2 + injections["L1_snr"]**2)

In [ ]:
plt.hist(network_snr_2det, bins=32, label="2", histtype="step", color="C0")
plt.hist(network_snr_3det, bins=32, label="3", histtype="step", color="C1")
plt.legend(title="\# detectors")
plt.xlabel("Network SNR")
plt.axvline(8, ls="--", color="k")
plt.text(8, 33, "SNR 8", rotation=0)
plt.show()

print("Median 3-detector network SNR: ", np.median(network_snr_3det))
print("Median 2-detector network SNR: ", np.median(network_snr_2det))

In [ ]:
paths = {
    "dynesty": {
        "3det": dynesty_3det_path,
        "2det": dynesty_2det_path,
    },
    "pocomc": {
        "3det": pocomc_3det_path,
        "2det": pocomc_2det_path,
    },
}

In [ ]:
paths

In [ ]:
data = {}
for sampler in paths.keys():
    data[sampler] = {}
    for det, path in paths[sampler].items():
        data[sampler][det] = {
            "sampling_time": np.nan * np.zeros(n_injections),
            "likelihood_evaluations": np.nan * np.zeros(n_injections),
            "n_samples": np.nan * np.zeros(n_injections),
            "log_evidence": np.nan * np.zeros(n_injections),
            "log_evidence_error": np.nan * np.zeros(n_injections),
        }
        for i in range(n_injections):
            try:
                result_file = next(path.glob(f"*injection_{i}.hdf5"))
            except StopIteration:
                print(f"Result file not found for injection {i} in {sampler} {det}")
                continue
            with h5py.File(result_file, "r") as f:
                if sampler == "dynesty":
                    data[sampler][det]["sampling_time"][i] = f["sampling_time"][()]
                data[sampler][det]["likelihood_evaluations"][i] = f["num_likelihood_evaluations"][()]
                data[sampler][det]["n_samples"][i] = len(f["posterior/mass_ratio"][()])
                data[sampler][det]["log_evidence"][i] = f["log_evidence"][()]
                data[sampler][det]["log_evidence_error"][i] = f["log_evidence_err"][()]
            
            if "pocomc" in sampler:
                # Get the sampling time from the sampling_time.dat file
                # This is a bit of a hack, but it works for now
                try:
                    timing_file = next(path.glob(f"sampling_time*_injection_{i}.dat"))
                except StopIteration:
                    continue
                data[sampler][det]["sampling_time"][i] = np.loadtxt(timing_file)

In [ ]:
figsize = plt.rcParams["figure.figsize"].copy()
figsize[1] = 2.5 * figsize[1]
fig, axs = plt.subplots(3, 1, figsize=figsize)

marker_kwargs = {
    "3det": dict(marker="o", s=5),
    "2det": dict(marker="^", s=5),
}
colours = {
    "pocomc": "C0",
    "dynesty": "C1",
}

for sampler, det_data in data.items():
    for det, vals in det_data.items():
        colour = colours[sampler]
        if det == "2det":
            colour = lighten_colour(colour, 0.5)
        axs[0].scatter(vals["likelihood_evaluations"], vals["sampling_time"] / 3600, label=f"{sampler} {det}", color=colour, **marker_kwargs[det])

        axs[1].scatter(vals["likelihood_evaluations"] / vals["n_samples"], vals["sampling_time"] / vals["n_samples"], label=f"{sampler} {det}", color=colour, **marker_kwargs[det])

axs[0].set_xlabel("Likelihood evaluations")
axs[0].set_ylabel("Sampling time [hours]")
axs[0].set_xscale("log")
axs[0].set_yscale("log")


axs[1].set_xlabel(r"Likelihood evaluations per sample")
axs[1].set_ylabel(r"Sampling time\\per sample [seconds]")
axs[1].set_xscale("log")
axs[1].set_yscale("log")

for det in ["3det", "2det"]:
    dynesty_evals_weighted = data["dynesty"][det]["likelihood_evaluations"] / data["dynesty"][det]["n_samples"]
    pocomc_evals_weighted = data["pocomc"][det]["likelihood_evaluations"] / data["pocomc"][det]["n_samples"]
    dynesty_time_weighted = data["dynesty"][det]["sampling_time"] / data["dynesty"][det]["n_samples"]
    pocomc_time_weighted = data["pocomc"][det]["sampling_time"] / data["pocomc"][det]["n_samples"]

    colour = "black"
    if det == "2det":
        colour = lighten_colour(colour, 0.5)
    axs[2].scatter(dynesty_evals_weighted / pocomc_evals_weighted, dynesty_time_weighted / pocomc_time_weighted, color=colour, **marker_kwargs[det])

    print(f"{det} mean ratio (evals): {np.nanmean(dynesty_evals_weighted / pocomc_evals_weighted):.2f} +/- {np.nanstd(dynesty_evals_weighted / pocomc_evals_weighted):.2f}")
    print(f"{det} mean ratio (time): {np.nanmean(dynesty_time_weighted / pocomc_time_weighted):.2f} +/- {np.nanstd(dynesty_time_weighted / pocomc_time_weighted):.2f}")

    print(f"{det} mean ratio (settings): {np.nanmean(dynesty_evals_weighted / pocomc_evals_weighted):.2f} +/- {np.nanstd(dynesty_evals_weighted / pocomc_evals_weighted):.2f}")
    print(f"{det} mean ratio (settings time): {np.nanmean(dynesty_time_weighted / pocomc_time_weighted):.2f} +/- {np.nanstd(dynesty_time_weighted / pocomc_time_weighted):.2f}")

axs[2].set_xlabel(r"\texttt{dynesty} / \texttt{pocomc} likelihood\\ evaluations per sample")
axs[2].set_ylabel(r"\texttt{dynesty} / \texttt{pocomc}\\ wall time per sample")

plt.tight_layout()

legend_handles = [
    plt.Line2D([0], [0], marker="o", color="w", label="3-detector", markerfacecolor="k", markersize=5),
    plt.Line2D([0], [0], marker="^", color="w", label="2-detector", markerfacecolor=lighten_colour("k", 0.5), markersize=5),
    plt.Line2D([0], [0], marker="s", color="w", label="pocomc", markerfacecolor="C0", markersize=5),
    plt.Line2D([0], [0], marker="s", color="w", label="dynesty", markerfacecolor="C1", markersize=5),
]
fig.legend(handles=legend_handles, loc="center", bbox_to_anchor=(0.5, -0.01), ncol=4)

fig.savefig("figures/sampling_time_vs_likelihood_evaluations.pdf", bbox_inches="tight")
plt.show()

In [ ]:
figsize = plt.rcParams["figure.figsize"].copy()
figsize[1] = 1.2 * figsize[1]
fig = plt.figure(figsize=figsize)

detector_ls = {
    "3det": {"ls": "-"},
    "2det": {"ls": "--"},
}
hist_kwargs = dict(
    bins=10,
    histtype="step",
)

labels = {
    "3det": "3 detectors",
    "2det": "2 detectors",
    "pocomc": r"\texttt{pocomc}",
    "dynesty": r"\texttt{dynesty}",
}


for sampler, det_data in data.items():
    for det, vals in det_data.items():
        colour = colours[sampler]
        if det == "2det":
            colour = lighten_colour(colour, 0.5)
        plt.hist(vals["n_samples"], label=f"{labels[sampler]} - {labels[det]}", color=colour, **detector_ls[det], **hist_kwargs)
legend = plt.legend(frameon=True, framealpha=1.0, fancybox=False, loc="upper right", fontsize=7)
legend.get_frame().set_linewidth(0.5)
plt.xlabel("I.I.D. samples")
plt.ylabel("Count")
plt.tight_layout()
fig.savefig("figures/n_samples.pdf", bbox_inches="tight")
plt.show()

Plot the difference in log-evidence between the two samplers

In [ ]:
figsize = plt.rcParams["figure.figsize"].copy()
figsize[1] = 2.3 * figsize[1]
fig, axs = plt.subplots(3, 1, figsize=figsize)

diff_2det = (data["dynesty"]["2det"]["log_evidence"] - data["pocomc"]["2det"]["log_evidence"])
diff_3det = data["dynesty"]["3det"]["log_evidence"] - data["pocomc"]["3det"]["log_evidence"]

relative_diff_2det = (data["dynesty"]["2det"]["log_evidence"] - data["pocomc"]["2det"]["log_evidence"]) / np.abs(data["dynesty"]["2det"]["log_evidence"])
relative_diff_3det = (data["dynesty"]["3det"]["log_evidence"] - data["pocomc"]["3det"]["log_evidence"]) / np.abs(data["dynesty"]["3det"]["log_evidence"])

error_diff_2det = (data["dynesty"]["2det"]["log_evidence_error"] - data["pocomc"]["2det"]["log_evidence_error"])
error_diff_3det = (data["dynesty"]["3det"]["log_evidence_error"] - data["pocomc"]["3det"]["log_evidence_error"])

axs[0].hist(diff_2det, bins=20, histtype="step", label="2", color=lighten_colour("k", 0.5))
axs[0].hist(diff_3det, bins=20, histtype="step", label="3", color="k")

axs[0].set_xlabel(r"$\Delta\ln Z$")
axs[0].set_ylabel("Count")

axs[1].hist(relative_diff_2det, bins=20, histtype="step", label="2", color=lighten_colour("k", 0.5))
axs[1].hist(relative_diff_3det, bins=20, histtype="step", label="3", color="k")

axs[1].set_xlabel(r"$\Delta \ln Z / \ln Z_{\texttt{dynesty}}$")
axs[1].set_ylabel("Count")

axs[2].hist(error_diff_2det, bins=20, histtype="step", label="2", color=lighten_colour("k", 0.5))
axs[2].hist(error_diff_3det, bins=20, histtype="step", label="3", color="k")
axs[2].set_xlabel(r"$\Delta \sigma[\ln Z]$")
axs[2].set_ylabel("Count")

axs[0].legend(title="\# detectors", loc="upper right")
plt.tight_layout()

fig.savefig("figures/log_evidence_differences.pdf", bbox_inches="tight")